In [1]:
#Various necessary imports
import numpy as np
import radmc3dPy
from radmc3dPy import analyze
from matplotlib import cm
from radmc3dPy.image import *
from radmc3dPy.analyze import *
from radmc3dPy.natconst import *

# Some natural constants
#
au  = 1.49598e13     # Astronomical Unit       [cm]
pc  = 3.08572e18     # Parsec                  [cm]
ms  = 1.98892e33     # Solar mass              [g]
ts  = 5.78e3         # Solar temperature       [K]
ls  = 3.8525e33      # Solar luminosity        [erg/s]
rs  = 6.96e10        # Solar radius            [cm]

# Custom constants

yr = 3.1536e7  # Seconds in a year
G = 6.6743e-8 # Gravitational constant in cgs
h = 6.626070e-27 #ergs s (Planck's constant)
c = 2.997925e10 #cm/s (Speed of light)
kb = 1.380649e-16 #erg/K (Boltzmann constant)
sigma = 5.67e-5
kb = 1.38e-16
hp = 6.626e-27
hok = hp/kb 
Rc = 8.314e7 # Gas constant

# Star parameters
#
mstar    = .44*ms
rstar    = 2*rs
tstar    = 3500
pstar    = np.array([0.,0.,0.])

#summon global vars
#(infall rate, truncation radius, mid-disk temp)
dMdt = ((1e-5/1)*ms)/yr
r_t = 30*au
midT = 100

In [2]:
#DEFINE X BOUNDS AND SIZE
x_start = .01*au
x_break = .1*au
x_end = 2000*au
x_end2 = 6000*au
x_hires_size = 80
x_lowres_size = 144
x_verylow_size = 20
x_size = x_hires_size + x_lowres_size +x_verylow_size

#DEFINE Y BOUNDS AND SIZE
y_start = 0
y_break = np.pi/2 - np.deg2rad(65)
y_end = np.pi/2
y_hires_size = 139
y_lowres_size = 10
y_size = y_hires_size + y_lowres_size

#DEFINE Z BOUNDS AND SIZE
z_start = 0
z_end = np.pi*2
z_size = 1

#DEFINE X, Y, Z COORDS
x_hires = np.geomspace(x_start, x_break, num=x_hires_size, endpoint=False)
x_lowres = np.geomspace(x_break, x_end, num=x_lowres_size, endpoint=False)
x_verylow = np.geomspace(x_end, x_end2, num=x_verylow_size+1)
x_vals = np.concatenate((x_hires, x_lowres, x_verylow))
x_vals_old = np.geomspace(x_start, x_end,num=x_size+1)

y_lowres = np.linspace(y_start, y_break, num=y_lowres_size, endpoint=False)
y_hires = np.linspace(y_break, y_end, num=y_hires_size+1)
y_vals = np.concatenate((y_lowres, y_hires))
y_vals_old = np.linspace(y_start, y_end, num=y_size+1)

z_vals = np.linspace(z_start, z_end, num=z_size+1)
# X Y Z for RHO...CANNOT BE +1
#NOTE: SEE cell where we write dust_density.inp for
#reason why we do the following
x_rvals = ( x_vals[1:] + x_vals[:-1]) / 2
y_rvals = ( y_vals[1:] + y_vals[:-1]) / 2
z_rvals = np.linspace(z_start, z_end, num=z_size)

In [3]:
#EXPERIMENTAL COPY OF ENVELOPE CODE
M = mstar # mass of inner body
r_c = r_t

def envelope(r, theta):
    #Abstracting r/r_c for easier calc
    x = r/r_c
    #Abstracting costheta
    mu = np.cos(theta)
    #Finding polynomial roots to calculate mu0 (cos of streamline theta, theta0)
    roots = np.roots([1, 0,-(1-x), -x*mu])
    for cur in roots:
        if cur>= 0:
            mu0= np.real(cur)
            break
    #Calculating density based on Eqn. 237 of LHM book
    rho = (( (dMdt) / (4*np.pi*np.sqrt(G*M*r**3)) )*(1 + np.cos(theta)/mu0)**(-1/2)*((np.cos(theta)/mu0)+ ((2*(mu0)**2)/(x)))**(-1))
    # Keplerian velocity calc
    v_theta = ((G*M)/r)**(1/2) * (mu0 - mu) * ((mu0+mu)/(mu0*(np.sin(theta))**2))**(1/2)
    #Calculation of envelope pressure (aka ram pressure)
    envP = rho * (v_theta)**2
    #Calculation of Z and R (Cylindrical coords)
    z = r*np.cos(theta)
    R = r*np.sin(theta)
    #Using our disk equation to solve for
    #Disk pressure (aka thermal pressure)
    #to facilitate exclusion
    #TEST/FIX THIS
    diskrho, c_s = diskdensityeqn(z, R, 5*au, Rc, 2.3, G, mstar, .16*ms, 10*au, r_t, midT)
    diskP = diskrho * (c_s)**2
    #Zeroing out envelope if streamline inside cavity/disk pressure exceeds envelope pressure
    if mu0 > np.cos(np.deg2rad(30)):
        return 0, 0
    if diskP > envP:
        return 0, 0
    #OPTIONALLY: You could zero out envelope past a certain radius.
    #We just keep the max envelope radius the same as the max grid radius.
    #if r > 3000*au:
    #    return 0
    else:
        return rho, envP

In [4]:
#DISK DENSITY FUNCTIONS

def Teqn(R, IR, basetemp):
    #Temperature 
    T = basetemp/((R/IR)**(1/2))
    return T

def Cseqn(Rc, T, mu):
    #Sound speed
    Cs = ((Rc*T)/(mu))**(1/2)
    return Cs

def Vphieqn(G, M, R):
    #Keplerian velocity
    Vphi = ((G*M)/R)**(1/2)
    return Vphi

def Heqn(Cs, R, Vphi):
    #Scale height
    H = (Cs*R)/(Vphi)
    return H

def Sigma0eqn(Md, R0, RT):
    #Surface density at chosen point
    Sigma0 = Md/(2*np.pi*R0*RT)
    return Sigma0

def Sigmaeqn(Sigma0, R, R0, RT):
    #Surface density
    Sigma = Sigma0 * (R/R0)**-1 * np.exp(-R/RT)
    return Sigma

def rho0eqn(Sigma, H):
    #Density at chosen point
    rho0 = Sigma / ( (2*np.pi)*H )
    return rho0

def rhoeqn(rho0, z, H):
    #Density at point
    rho = rho0 * np.exp(-(z**2)/(2*H**2))
    return rho

def diskdensityeqn(z, R, IR, Rc, mu, G, M, Md, R0, RT, basetemp):
    """
    z = distance from midplane
    R = distance from central axis
    IR = inner radius of the disk
    Rc = gas constant
    mu = grams/mol for the dust. for example, approx 2.3 for high concentration of molecular H.
    G = gravitational constant
    M = mass of inner body (star)
    Md = mass of the disk
    R0 = radius where we set the sigma constant. ideally towards middle, between disk start and end.
    RT = truncation radius where the disk ends
    basetemp = temperature at R0, affects puffiness of disk
    """
    #We just run through all of the above calculations, hitting each one
    T = Teqn(R, IR, basetemp) #IR
    Cs = Cseqn(Rc, T, mu) # RC MU
    Vphi = Vphieqn(G, M, R) # G M
    H = Heqn(Cs, R, Vphi) 
    Sigma0 = Sigma0eqn(Md, R0, RT) #Md, R0, RT
    Sigma = Sigmaeqn(Sigma0, R, R0, RT)
    rho0 = rho0eqn(Sigma, H) 
    rho = rhoeqn(rho0, z, H)
    
    if np.isnan(rho) == True:
        return 0, 0
    else:
        return rho, Cs

In [5]:
def disk(r, theta):
    z = r*np.cos(theta)
    R = r*np.sin(theta)
    diskrho, Cs = diskdensityeqn(z, R, 5*au, Rc, 2.3, G, mstar, .16*ms, 10*au, r_t, midT)
    diskP = diskrho * (Cs)**2
    envP = envelope(r, theta)[1]

    if envP > diskP:
        return 0
    else:
        return diskrho

In [6]:
#writing dust_density.inp
#
#NOTE: Each set of for statements is for a different 
#dust species. 
#ALSO, EXTREMELY IMPORTANT NOTE:
#Due to the wackiness of RADMC3D, when dealing with weird grid cells, 
#we must realize that there's a distinction between cell EDGES and CENTERS
#I have referred to these, in the gridding, as X, Y, Z vals for EDGES
#and Xr, Yr, Zr vals for CENTERS
#If we do not calculate densities in the centers (using r_vals), 
#we either over or underestimate all density values (think of trapezoidal rule)

#If I remember correctly, the RADMC tutorial files may not do this correctly. 

with open('dust_density.inp','w+') as f:
    f.write('1\n')                       # Format number
    f.write('%d\n'%(x_size*y_size*z_size))           # Nr of cells
    f.write('4\n') # Nr of dust species
    for theta in y_rvals:
        for rval in x_rvals:
            f.write('%13.6e\n'%(disk(rval, theta)))
    for theta in y_rvals:
        for rval in x_rvals:
            f.write('%13.6e\n'%(disk(rval, theta)))
    for theta in y_rvals:
        for rval in x_rvals:
            f.write('%13.6e\n'%(envelope(rval, theta)[0]))
    for theta in y_rvals:
        for rval in x_rvals:
            f.write('%13.6e\n'%(envelope(rval, theta)[0]))